In [3]:
import pandas as pd
import numpy as np
from sklearn.decomposition import TruncatedSVD

train = pd.read_csv('train.csv')
train['count'] = train.groupby('movie_id')['movie_id'].transform('count')
train['weight'] = train['rating'] * train['count']
ratings_crosstab = train.pivot_table(values='rating', index='user_id', columns= 'movie_id', fill_value=0)
X = ratings_crosstab.values.T
SVD = TruncatedSVD(n_components= 22, random_state= 17)
resultant_matrix = SVD.fit_transform(X)
corr_mat = np.corrcoef(resultant_matrix)
movies_names = ratings_crosstab.columns
movie_list = list(movies_names)
with open('recommendations.csv', 'r') as file:
    x = file.readlines()
f = open('ans.csv','w')
x = map(int,x)

for user_id in x:
#     user_movies = train[['movie_id','count','rating']][train.user_id == user_id]
#     user_movies.sort_values(['rating','count'], ascending = False, inplace = True)


    user_movies = train[['movie_id','weight']][train.user_id == user_id]
    user_movies.sort_values(['weight'], ascending = False, inplace = True)

    
    recommend = []
    for movie in user_movies['movie_id'].values:
        index = movie_list.index(movie)
        corr_movie = corr_mat[index]
        mfilter = (corr_movie < 1.0) & (corr_movie > 0.911)
        name = movies_names[mfilter]
        corrvalue = corr_movie[mfilter]
        movies_df = pd.DataFrame({'name':name, 'corrvalue':corrvalue}).sort_values('corrvalue', ascending=False)
        for m in movies_df['name'].values:
            if (m not in user_movies['movie_id'].values) and (m not in recommend):
                recommend.append(m)
            if len(recommend) >= 10:
                break
        if len(recommend) >= 10:
            break
    recommend = map(str, recommend)
    f.write(str(user_id)+', '+ str(','.join(recommend))+'\n')
f.close()